In [1]:
import sys
print(sys.path)

['/Users/lucasmeng/PyBismarck', '/usr/local/bin/python', '/Users/lucasmeng/PyBismarck', '/Users/lucasmeng/anaconda3/envs/py37/lib/python37.zip', '/Users/lucasmeng/anaconda3/envs/py37/lib/python3.7', '/Users/lucasmeng/anaconda3/envs/py37/lib/python3.7/lib-dynload', '', '/Users/lucasmeng/anaconda3/envs/py37/lib/python3.7/site-packages', '/Users/lucasmeng/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/extensions', '/Users/lucasmeng/.ipython']


In [2]:
import psycopg2
import pandas as pd
import numpy as np
import time

In [3]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [4]:
connection = psycopg2.connect(database = "test_bismarck",
                             user = "bismarckvm",
                             host= "127.0.0.1",
                             port = "5432")

In [5]:
from warnings import filterwarnings
filterwarnings('ignore')

In [27]:
start = time.time()

connection = psycopg2.connect(database = "test_bismarck",
                             user = "bismarckvm",
                             host= "127.0.0.1",
                             port = "5432")

iterations = 20
limit = 50000
epochs = 581012 // limit + 1
clf = LogisticRegression(random_state=0, solver='sag', penalty='l2', max_iter=1, warm_start=True)

for iter in range(iterations):
    loss = 0.0
    for e in range(epochs):
        offset = e * limit
        select_query = "select * from forest limit " + str(limit) + " offset " + str(offset) + ";"
        forest_df = pd.read_sql(select_query, connection)

        X = list(forest_df["vec"])
        X = np.array(X)
        y = forest_df["labeli"].values
        clf.fit(X, y)
        w = (clf.coef_).reshape(-1, len(clf.coef_))
        loss += sum(np.log(1 + np.exp(np.multiply((np.dot(X, w) + clf.intercept_), -y.reshape(len(y),-1)))))
#         print(loss)
        
    print("Epoch ", iter, loss)
    print("--- %s seconds ---" % (time.time() - start))

print("--- %s seconds ---" % (time.time() - start))

Epoch  0 [305145.77190648]
--- 39.0962700843811 seconds ---
Epoch  1 [298974.97627127]
--- 77.6692099571228 seconds ---
Epoch  2 [299068.3198017]
--- 116.08256602287292 seconds ---
Epoch  3 [299133.75694716]
--- 155.05247282981873 seconds ---
Epoch  4 [298936.42298659]
--- 193.99852180480957 seconds ---
Epoch  5 [299200.3929031]
--- 232.62483096122742 seconds ---
Epoch  6 [299016.14068494]
--- 270.9761381149292 seconds ---
Epoch  7 [299080.2562]
--- 309.7645399570465 seconds ---
Epoch  8 [298770.17704546]
--- 348.1434681415558 seconds ---
Epoch  9 [298554.56533713]
--- 386.46035504341125 seconds ---
Epoch  10 [299417.27796534]
--- 425.1483941078186 seconds ---
Epoch  11 [299547.58178128]
--- 464.6201648712158 seconds ---
Epoch  12 [298354.79948822]
--- 503.2972888946533 seconds ---
Epoch  13 [299202.70480805]
--- 543.927169084549 seconds ---
Epoch  14 [299262.19768488]
--- 588.0253570079803 seconds ---
Epoch  15 [298921.41803801]
--- 631.0649271011353 seconds ---
Epoch  16 [299129.6506

In [ ]:
from sklearn.svm import SVC

In [ ]:
start = time.time()

connection = psycopg2.connect(database = "test_bismarck",
                             user = "bismarckvm",
                             host= "127.0.0.1",
                             port = "5432")

iterations = 20
limit = 50000
epochs = 581012 // limit + 1
clf = SVC(random_state=0, probability=True, max_iter=1, warm_start=True)

for iter in range(iterations):
    loss = 0.0
    for e in range(epochs):
        offset = e * limit
        select_query = "select * from forest limit " + str(limit) + " offset " + str(offset) + ";"
        forest_df = pd.read_sql(select_query, connection)

        X = list(forest_df["vec"])
        X = np.array(X)
        y = forest_df["labeli"].values
        clf.fit(X, y)
        w = (clf.coef_).reshape(-1, len(clf.coef_))
        loss += sum(np.log(1 + np.exp(np.multiply((np.dot(X, w) + clf.intercept_), -y.reshape(len(y),-1)))))
#         print(loss)
        
    print("Epoch ", iter, loss)
    print("--- %s seconds ---" % (time.time() - start))

print("--- %s seconds ---" % (time.time() - start))

In [25]:
from sklearn.linear_model import SGDClassifier

In [ ]:
start = time.time()

connection = psycopg2.connect(database = "test_bismarck",
                             user = "bismarckvm",
                             host= "127.0.0.1",
                             port = "5432")

iterations = 20
limit = 50000
epochs = 581012 // limit + 1
clf = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-2, max_iter=1, random_state=0, warm_start=True)
w = np.zeros(54)
b = [0.0]

for iter in range(iterations):
    loss = 0.0
    for e in range(epochs):
        offset = e * limit
        select_query = "select * from forest limit " + str(limit) + " offset " + str(offset) + ";"
        forest_df = pd.read_sql(select_query, connection)

        X = list(forest_df["vec"])
        X = np.array(X)
        y = forest_df["labeli"].values
        if e != 0:
            clf.coef_[0] = w
            clf.intercept_[0] = b
#             print(clf.coef_)
        clf.partial_fit(X, y, classes=[-1, 1])
#         w = (clf.coef_).reshape(-1, len(clf.coef_))
        w = clf.coef_[0]
        b = clf.intercept_[0]
#         print(w)
        loss += clf.score(X, y)
#         print(loss)
        
    print("Epoch ", iter, loss/epochs)

print("--- %s seconds ---" % (time.time() - start))

In [15]:
select_query = "select * from forest limit " + str(5000) + " offset " + str(0) + ";"
forest_df = pd.read_sql(select_query, connection)

X = list(forest_df["vec"])
X = np.array(X)
y = forest_df["labeli"].values
clf.fit(X, y)
w = (clf.coef_).reshape(-1, len(clf.coef_))
loss = np.log(1 + np.exp(np.multiply((np.dot(X, w) + clf.intercept_), -y.reshape(len(y),-1))))

In [16]:
loss

array([[0.00663589],
       [1.28014694],
       [0.8918104 ],
       ...,
       [0.5011238 ],
       [0.3279529 ],
       [0.06660356]])

In [17]:
state = [[1,2,3],[4,5,6],[7,8,9]]

In [21]:
s = np.array(state)

In [22]:
s[:, -1]

array([3, 6, 9])

In [23]:
s[:, :-1]

array([[1, 2],
       [4, 5],
       [7, 8]])

In [20]:
[el[-1] for el in state]

[3, 6, 9]

In [15]:
np.array(state[:][:-1])

array([[1, 2, 3],
       [2, 3, 4]])